In [1]:
def _target_encoding(xtr, xholdout, xte, to_enc, nfolds, num_iterations, min_samples_leaf):
    
    '''
    кодирует категории целевым признаком 
    
    Параметры:
        1) xtr - трейн
        2) xholdout - отложенная
        3) xte - тест
        4) to_enc-колонки, которые кодируются
        5) nfolds- число фолдов
        6) num_iterations = число итераций кодирования
    '''
    # списки с кодированиями
    encs_tr, encs_holdout, encs_te = [], [], []
    # проходим по числу итераций
    for _ in tqdm_notebook(range(num_iterations)):
        # задаем датафреймы, которые будем заполнять
        enc_tr = pd.DataFrame(index = xtr.index, columns = to_enc).fillna(0.0)
        enc_holdout = pd.DataFrame(index = xholdout.index, columns = to_enc).fillna(0.0)
        enc_te = pd.DataFrame(index = xte.index, columns = to_enc).fillna(0.0)
        # запускаем валидацию
        for i, (tr_idx, val_idx) in enumerate(KFold(nfolds, random_state = _).split(xtr)):
            # получаем датафрейм, на котором будет оцениваться средний таргет
            _df_tr = pd.concat([xtr.iloc[tr_idx], ytr.iloc[tr_idx]], 1)
            # считаем средний таргет на всем трейн датасете
            y_tr_mean = ytr.iloc[tr_idx].mean()
            # проходим по колонкам, которые будут кодироваться
            for col in to_enc:  
                # словарь с мапингом среднего таргета по категорям,
                # (ограниченный минимальным размером категории, для которого будет считаться средний таргет)
                _ddf =_df_tr.groupby(col)['target'].agg({'mean', 'count'})
                d_map = _ddf[_ddf['count']>=min_samples_leaf]['mean'].to_dict()
                # применяем кодирование на валидации + шум
                enc_tr.loc[xtr.iloc[val_idx].index, col] = xtr.iloc[val_idx][col].map(d_map)\
                                                            + np.random.normal(y_tr_mean/100, y_tr_mean/500, size = len(val_idx))
                # применяем кодирование на отложенной, нормируя на число фолдов + шум
                enc_holdout.loc[:, col] += (xholdout[col].map(d_map)\
                                            + np.random.normal(y_tr_mean/100, y_tr_mean/500, size = len(xholdout)))\
                                            / nfolds 
                # применяем кодирование на тесте, нормируя на число фолдов + шум
                enc_te.loc[:, col] += (xte[col].map(d_map)\
                                            + np.random.normal(y_tr_mean/100, y_tr_mean/500, size = len(xte)))\
                                            / nfolds 
        # коллекционируем кодирования
        encs_tr.append(enc_tr)
        encs_holdout.append(enc_holdout)
        encs_te.append(enc_te)
        
    # усредняем коллекции кодирований
    enc_final_tr = sum(encs_tr) / len(encs_tr)  
    enc_final_holdout = sum(encs_holdout) / len(encs_holdout)   
    enc_final_te = sum(encs_te) / len(encs_te)  
    
    return (enc_final_tr, enc_final_holdout, enc_final_te)


In [2]:
def _do_stacking(estimators, xtr, xholdout, xte, nfolds, num_iterations):
    
    '''
    кодирует категории целевым признаком 
    
    Параметры:
        1) xtr - трейн
        2) xholdout - отложенная
        3) xte - тест
        4) estimators - базовые модели
        5) nfolds- число фолдов
        6) num_iterations = число итераций кодирования
    '''
    # списки с метапризнаками
    oofs_tr, oofs_holdout, oofs_te = [], [], []
    # проходим по числу итераций
    for _ in tqdm_notebook(range(num_iterations)):
        # задаем датафреймы, которые будем заполнять
        oof_tr = pd.DataFrame(index = xtr.index, columns = ['oof_predictions']).fillna(0.0)
        oof_holdout = pd.DataFrame(index = xholdout.index, columns = ['oof_predictions']).fillna(0.0)
        oof_te = pd.DataFrame(index = xte.index, columns = ['oof_predictions']).fillna(0.0)         
        # запускаем валидацию
        for i, (tr_idx, val_idx) in enumerate(KFold(nfolds, random_state = _).split(xtr)): 
            # проходим по моделям
            for estimaor in estimators:
                # фитим на трейне
                estimator.fit(xtr.iloc[tr_idx], ytr.iloc[tr_idx])
                # предсказываем валидацию
                oof_tr.loc[xtr.iloc[val_idx].index, 'oof_predictions'] = estimator.predict_proba(xtr.iloc[val_idx])[:, 1]
                # усредняем предсказания каждого фолда на отложенной и на тесте
                oof_holdout.loc[:, 'oof_predictions'] += estimator.predict_proba(xholdout)[:, 1] / nfolds
                oof_te.loc[:, 'oof_predictions'] += estimator.predict_proba(xte)[:, 1] / nfolds
        # коллекционируем метапризнаки
        oofs_tr.append(oof_tr)
        oofs_holdout.append(oof_holdout)
        oofs_te.append(oof_te)
        
    # усредняем списки с метапризнаками
    oofs_final_tr = sum(oofs_tr) / len(oofs_tr)  
    oofs_final_holdout = sum(oofs_holdout) / len(oofs_holdout)   
    oofs_final_te = sum(oofs_te) / len(oofs_te)  
    
    return (oofs_final_tr, oofs_final_holdout, oofs_final_te)

In [3]:
def _tune_lgb(xtr, xholdout, ytr, yholdout, metric, eval_metric, paramGrid, cv, early_stopping_rounds, type_task):
    '''
    Оптимизирует гиперпараметры lightgbm
    
    Параметры:
        1) xtr - признаки на трейне
        2) xholdout - признаки на отложенной
        3) ytr - таргет на трейне
        4) yholdout - таргет на отложенной
        5) metric - оптимизируемая метрика качества
        6) eval_metric - метрика для ранней остановки
        7) paramGrid - сетка перебора
        8) cv - схема валидации
        9) early_stopping_rounds - ранняя остановка
        10) type_task - тип задачи
        
    Возвращает: лучшую метрику, лучшие параметры
    '''
    
    if type_task == 'classification':
        model = lgb.sklearn.LGBMClassifier(random_state =SEED)
    else:
        model = lgb.sklearn.LGBMRegressor(random_state =SEED)
            
    if (early_stopping_rounds is not None) and (eval_metric is not None):
        fit_params={"early_stopping_rounds":early_stopping_rounds, 
                    "eval_metric" : eval_metric, 
                    "eval_set" : [[xholdout, yholdout]]}
        gridsearch = GridSearchCV(model, paramGrid, verbose=0,             
             cv=cv, scoring = metric)

        gridsearch.fit(xtr, ytr, **fit_params)
        
        return (gridsearch.best_score_, gridsearch.best_params_)
    elif (early_stopping_rounds is None) and (eval_metric is None):
        gridsearch = GridSearchCV(model, paramGrid, verbose=0,             
             cv=cv, scoring = metric)
        gridsearch.fit(xtr, ytr)
        
        return (gridsearch.best_score_, gridsearch.best_params_)
    else:
        raise ValueError('early stopping и eval metric должны быть None или not None')

In [4]:
def _select_features(clf, xtr, ytr, cv, metric):
    
    '''
    Отбирает признаки (метрика = roc auc)
    
    Параметры:
        1) clf - модель
        2) xtr - признаки
        3) ytr - таргет
        4) cv - схема валидации
        5) early_stopping - кол-во итераций без улучшения метрики для остановки первой стадии отбора
        6) min_diff_continue - минимальное кол-во отобранных признаков за итерацию для продолжения второй стадии отбора
    
    Возвращает:
        лучшее значение метрики, лучший набор признаков
    '''
    
    # валидируем каждый признак
    scores = []
    for i in tqdm_notebook(range(xtr.shape[1])):
        scores.append(cross_validate(clf, xtr[:, i].reshape(-1,1),\
                                     ytr, scoring ='roc_auc', cv = cv)['test_score'].mean())
    # сортируем индивидуальные скоры в порядке убывания
    order = np.argsort(scores)[::-1]
    # лучший скор, лучшие признаки, признаки для проверки, счетчик ранней остановки
    best_score, best_features, to_drop, = .5, [], []
    # по признакам в порядке убывания индивидуальных скоров
    for i in tqdm_notebook(order):
        # добавляем признак в список лучших
        current_features = best_features+[i]
        # считаем валидацю
        mean_score = cross_validate(clf, xtr[:, current_features],\
                                     ytr, scoring =metric, cv = cv)['test_score'].mean()
        # если есть улучшение
        if mean_score > best_score:
            # обновляем лучший скор
            best_score = mean_score
            # обновляем лучшие признаки
            best_features = current_features            
            
        # если улчшения нет
        else:
            # добавляем признак в кандидаты на удаление
            to_drop.append(i)
            # обновляем счетчик
    print('лучшее значение метрики = {}'.format(best_score))
    # списки на удаление признаков ДО и ПОСЛЕ отбора
    to_drop_before = to_drop
    to_drop_after = []
    # запускаем бесконечный цикл
    while True:
        # добавляем последовательно признаки из кандидатов на удаление
        for i in tqdm_notebook(to_drop_before):
            current_features = best_features+[i]
            mean_score = cross_validate(clf, xtr[:, current_features],\
                                     ytr, scoring =metric, cv = cv)['test_score'].mean()
            # если есть улучшение скора
            if mean_score > best_score:
                # обновляем лучший скор
                best_score = mean_score
                # обновляем лучшие признаки
                best_features = current_features            
            else:
                # добавляем признак в кандидаты на удаление ПОСЛЕ
                to_drop_after.append(i)
                
        # если списки ДО и ПОСЛЕ отличаются меньше, чем на 5 признаков 
        if len(to_drop_before) == len(to_drop_after):
            # останавливаем отбор
            break
        else:
            # ДО --> ПОСЛЕ, ПОСЛЕ - пустой
            to_drop_before = to_drop_after
            to_drop_after = []
        print('лучшее значение метрики = {}'.format(best_score))
            
    return (best_score, best_features)

In [5]:
def _exclude_outliers_iqr(series):
    '''
    помечает выбросы (найденные с помощью интерквартильного размаха)
    '''
    q25, q75 = series.quantile([.25, .75])
    iqr = q75-q25
    return series.between(q25-1.5*iqr, q75+1.5*iqr)

In [6]:
def _identify_different_distributions(xtr, xholdout, alpha):
    '''
    проверят идентичность распределений признаков в трейне и в отложенной частях
    (тест колмогорова-смирнова)
    
    Возвращает: True, если различия есть,иначе - False
    '''
    
    flags = []
    for col in sites:
        ser1 = xtr[col]
        ser2 = xholdout[col]
        if stats.ks_2samp(ser1, ser2)[1] < alpha:
            flags.append(True)
        else:
            flags.append(False)        
    flags = np.array(flags)    
    if flags.sum() > 0:
        return True
    else:
        return False

In [7]:
def _train_holdout_split(X, y, holdout_share, seed, stratify, shuffle):
    '''
    делит данные на трейн и отложенную части
    
    парметры:
        1) X - признаки
        2) y - таргет
        3) holdout_share - доля отложенной выборки
        4) seed - генератор случайных чисел
        5) stratify - стратификация таргета (TRUE, FALSE)
        6) shuffle - перемешивание данных (TRUE, FALSE)
        
    возвращает: признаки трейн, признаки отоженная, таргет трейн, таргет отложенная
    '''
    if stratify:
        if shuffle:
            xtr, xholdout, ytr, yholdout = train_test_split(X, y, test_size = holdout_share, stratify = y, random_state = seed)
        else:
            xtr, xholdout, ytr, yholdout = train_test_split(X, y, test_size = holdout_share, stratify = y)
            
    else:
        if shuffle:
            xtr, xholdout, ytr, yholdout = train_test_split(X, y, test_size = holdout_share, random_state = seed)
        else:
            xtr, xholdout, ytr, yholdout = train_test_split(X, y, test_size = holdout_share)
            
    return (xtr, xholdout, ytr, yholdout)
            

In [8]:
def _get_data_report(df_tr):
    '''
    1) Удаляет повторяющиеся строки
    2) преобразует даты (по возможности)
    3) делает отчет по датафрейму
    '''
    
    duplicates_mask = df_tr.duplicated()
    df_tr = df_tr[~duplicates_mask]
    L = []
    print('Дубликаты: % дубликатов строк = {:.1%}\n'.format(duplicates_mask.mean()))
    for col in df_tr.columns[:-1]:
        try:
            df_tr[col] = df_tr[col].astype('datetime64')
        except:
            pass
        ser = df_tr[col]
        _dtype = ser.dtype
        _nuniques = ser.nunique()
        if _dtype not in  ('object', 'datetime64[ns]'):
            mn, mx = ser.min(), ser.max()
        else:
            mn, mx = np.nan, np.nan
        L.append((col, _dtype, _nuniques, ser.isna().mean().round(2), mn, mx))
        
    report_df = pd.DataFrame.from_records(L)\
                  .rename(columns = {0:'признак', 1:'тип', 2:'число_уник_зн',\
                                     3:'доля nan', 4:'миниум', 5:'максимум'})
    print('Таблица:')
    print(report_df)
    
    return df_tr